In [3]:
from polyname import check_database
from polyname import remove_duplicates
from polyname import search_names
from polyname import polymer_extract
from polyname import strip_poly
from polyname import get_pubchem_smiles
from polyname import get_cirpy_smiles
from polyname import get_pubchem_names
from polyname import get_cirpy_names
from polyname import lowercase
import pandas as pd


In [4]:
#POLYMER EXTRACTION STEP
#Input : chemical names outputted by the extraction step
all_names_path = "polyname_example/all_names.txt"
df = pd.read_table(all_names_path)
all_names_list = df["All_names"].to_list()
print(all_names_list)

#isolate the polymers using a list of chemical entities
polymer_list = polymer_extract(all_names_list)
#print DF
#convert it to dataframe and write to a textfile with all isolated polymer names
data = pd.DataFrame (polymer_list, columns = ['Isolated_poly_names'])
print(data)
data.to_csv("polyname_example/poly_exam_final_names.txt", index = False, header = False)

['Polypropylene', 'Pla', 'Landðll', 'Phas', 'Packaging.8', 'Poly(3-hydroxybutyrate)\npoly(3-hydroxybutyrate-co-3-hydroxyvalerate)', 'Biopol', 'Gelatins', 'Plastic fate', 'Pga', 'Polyesters', 'Phbv', 'Vinyl', 'Poly(glycolic acid)', 'Poly(p-dioxanone)', 'Dexon', 'Birmingham b4 7et', 'Poly(a-ester)s', 'Oxygen', '70·7\n3·2', 'Nickel', 'Methane', 'Poly(hydroxyalkanoate)s', 'Petroleum', 'Polyanhydrides', 'Plla', 'Polycarbonates', 'T ighe', 'Polyester', 'Polystyrene', 'Pva', 'Polydioxanones', 'Pcl', 'Lactomer', 'Poly(lactic\nacid)', 'Polyoleðns', 'Britain', 'Poly(caprolactone)', 'Naphtha', 'Beneðts', 'Poly(vinyl alcohol)', 'Pira', 'Polyethylene', 'Vicryl', 'P3hb', 'Pgla', 'Pack', 'Polydioxanone']
                                  Isolated_poly_names
0                                     Poly(propylene)
1   Poly(3-hydroxybutyrate\npoly3-hydroxybutyrate-...
2                                         Poly(ester)
3                                 Poly(glycolic acid)
4                              

In [6]:
#CHECK DATABASE STEP
#Input : database file name and extracted polymers file (poly_exam_final_names.txt)
database_path = "polyname_example/poly_name_smiles_database.csv"
all_polymers = "polyname_example/poly_exam_final_names.txt"
#check database for each polymer name and remove duplicates
name,smile = check_database(all_polymers,database_path)
final_name,final_smile = remove_duplicates(name,smile)

#add polymer names and monomer smiles 
df = pd.DataFrame(final_name, columns = ["Polymer Names"])
df.insert(1,'Monomer Smiles', final_smile)
print(df)
df.to_csv('polyname_example/database_extract.csv', index = None)

#return a list with polymer names not found in database, Input: the all polymers names and final names found in DB
search_name = search_names(all_polymers,final_name)
print(search_name)
#write the names not found in DB into a df2
lowercase_poly = lowercase(search_name)
df2 = pd.DataFrame(lowercase_poly, columns = ["polymername"])
df2.to_csv('polyname_example/polymer_extract_still.csv', index = None)

['"Poly(3-hydroxybutyrate', 'poly3-hydroxybutyrate-co-3-hydroxyvalerate)"', 'Poly(ester)', 'Poly(dioxanone)', 'Poly(hydroxyalkanoate)', 'Poly(anhydride)', 'Poly(a-ester)', 'Poly(dioxanone)', 'Poly(glycolic acid)', 'Poly(carbonate)', 'Poly(styrene)', 'Poly(oleðn)', '"Poly(3-hydroxybutyrate', 'poly3-hydroxybutyrate-co-3-hydroxyvalerate)"', 'Poly(propylene)', '"Poly(lactic', 'acid)"', 'Poly(ethylene)', 'Poly(caprolactone)', 'Poly(p-dioxanone)', 'Poly(ester)', 'Poly(vinyl alcohol)']
     Polymer Names Monomer Smiles
0   Poly(ethylene)            C=C
1  Poly(propylene)           C=CC
2    Poly(styrene)    C=Cc1ccccc1
['"Poly(3-hydroxybutyrate', 'poly3-hydroxybutyrate-co-3-hydroxyvalerate)"', 'Poly(ester)', 'Poly(dioxanone)', 'Poly(hydroxyalkanoate)', 'Poly(anhydride)', 'Poly(a-ester)', 'Poly(glycolic acid)', 'Poly(carbonate)', 'Poly(styrene)', 'Poly(oleðn)', 'Poly(propylene)', '"Poly(lactic', 'acid)"', 'Poly(ethylene)', 'Poly(caprolactone)', 'Poly(p-dioxanone)', 'Poly(vinyl alcohol)']


In [3]:
#EXTRACT MONOMER SMILES NAMES come back to this
#Extract monomer smiles from the polymers that still need to be extracted 
polymers_to_extract = 'polyname_example/polymer_extract_still.csv'
#read textfile into dataframe
data2 = pd.read_table(polymers_to_extract)
print(data2)
#get unique polymer names 
dfnames2 = pd.DataFrame(data2.polymername.unique(),columns=['polyname'])
print(dfnames2)
stripstrings = [['poly(',')'],['poly[(',')]'],['poly[',']'],['poly{','}'],['poly-','']]
dfnames2['stripped_monomer_name'] = dfnames2.apply(lambda row: strip_poly(row,stripstrings),axis=1)
print(dfnames2)
#pubchem is busy
monomer_names2 = dfnames2.apply(lambda row: strip_poly(row,stripstrings),axis=1)
dfnames2['Pubchem Smiles'] = monomer_names2.apply(lambda row: get_pubchem_smiles(row))
dfnames2['Cirpy Monomer Smiles'] = monomer_names2.apply(lambda row: get_cirpy_smiles(row))
dfnames2['Pubchem Names'] = monomer_names2.apply(lambda row: get_pubchem_names(row))
dfnames2['Cirpy Monomer Iupac Names'] = monomer_names2.apply(lambda row: get_cirpy_names(row))
print(dfnames2)
dfnames2.to_csv("polyname_example/pubchem_cirpy_polymer_extract.csv", index = False)

                                    polymername
0                        poly(3-hydroxybutyrate
1   poly3-hydroxybutyrate-co-3-hydroxyvalerate)
2                                   poly(ester)
3                               poly(dioxanone)
4                        poly(hydroxyalkanoate)
5                               poly(anhydride)
6                                 poly(a-ester)
7                           poly(glycolic acid)
8                               poly(carbonate)
9                                 poly(styrene)
10                                  poly(oleðn)
11                              poly(propylene)
12                            poly(lactic acid)
13                               poly(ethylene)
14                           poly(caprolactone)
15                            poly(p-dioxanone)
16                          poly(vinyl alcohol)
                                       polyname
0                        poly(3-hydroxybutyrate
1   poly3-hydroxybutyrate-co-3-hydroxyva